Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import cPickle as pickle
import numpy as np
import tensorflow as tf

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print 'Training set', train_dataset.shape, train_labels.shape
  print 'Validation set', valid_dataset.shape, valid_labels.shape
  print 'Test set', test_dataset.shape, test_labels.shape

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (18724, 28, 28) (18724,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print 'Training set', train_dataset.shape, train_labels.shape
print 'Validation set', valid_dataset.shape, valid_labels.shape
print 'Test set', test_dataset.shape, test_labels.shape

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (18724, 28, 28, 1) (18724, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. 
-----------
Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [26]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  ## conv layers
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1)) ## feat map 1 -> 16
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1)) ## feat map 16 -> 16
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))

  ## fc layer
  ## 2 conv's reduced map size to 1/4 img x 1/4 img x depth 16
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size / 4 * image_size / 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  ## readout layer
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))  ## num hidden nodes -> num labels
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()  ## batch, height, width, depth
    print shape
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])

    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

[16, 7, 7, 16]
[10000, 7, 7, 16]
[18724, 7, 7, 16]


** Run it **

In [27]:
num_steps = 2001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized\n==============="
  for step in xrange(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 200 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.2f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.2f%%" % accuracy(valid_prediction.eval(), valid_labels)
      print "---------------------------"
  print "Test accuracy: %.2f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 3.90634
Minibatch accuracy: 0.00%
Validation accuracy: 10.16%
---------------------------
Minibatch loss at step 200 : 1.06837
Minibatch accuracy: 81.25%
Validation accuracy: 74.79%
---------------------------
Minibatch loss at step 400 : 0.670855
Minibatch accuracy: 81.25%
Validation accuracy: 79.49%
---------------------------
Minibatch loss at step 600 : 0.941572
Minibatch accuracy: 68.75%
Validation accuracy: 80.51%
---------------------------
Minibatch loss at step 800 : 0.643219
Minibatch accuracy: 87.50%
Validation accuracy: 81.72%
---------------------------
Minibatch loss at step 1000 : 0.735526
Minibatch accuracy: 75.00%
Validation accuracy: 82.56%
---------------------------
Minibatch loss at step 1200 : 0.773947
Minibatch accuracy: 75.00%
Validation accuracy: 82.56%
---------------------------
Minibatch loss at step 1400 : 0.637286
Minibatch accuracy: 87.50%
Validation accuracy: 83.35%
---------------------------
Minibatch loss at step

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides with a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

** Two 4x4 conv w/ max pooling (16@14x14, 32@7x7), fc 128 **
> steps 6001, decay (.05, 100, .96), Test accuracy: **94.08**%<br>
> steps 12499, decay (.05, 200, .96), Test accuracy: **94.89**%<br>
> steps 12499, decay (.04, 100, .96), Test accuracy: **93.89**%<br>
> steps 6001, decay (.04, 200, .96), Test accuracy: **94.22**%<br>
> steps 6001, decay (.05, 200, .96), Test accuracy: **94.45**%<br>

    12001 0.00431673
    Minibatch loss at step 12000 : 0.0330466
    Minibatch accuracy: 100.00%
    Validation accuracy: 89.47%
    Test accuracy: 94.89%

** Two 3x3 conv w/ max pooling (6@14x14, 16@7x7), fc 256 **
> steps 12499, decay (.05, 200, .96), Test accuracy: **94.46**%<br>

    12001 0.00431673
    Minibatch loss at step 12000 : 0.0192785
    Minibatch accuracy: 100.00%
    Validation accuracy: 88.44%
    Test accuracy: 94.46%

** Two 5x5 conv w/ max pooling (6@14x14, 16@7x7), fc 512 **
> steps 6001, decay (.05, 200, .96), Test accuracy: **94.34**%<br>

    6001 0.0146899
    Minibatch loss at step 6000 : 0.630632
    Minibatch accuracy: 68.75%
    Validation accuracy: 88.40%
    Test accuracy: 94.34%

---

** Helper functions **

In [12]:
## Weight Initialization
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape, constant=0.1):
  initial = tf.constant(constant, shape=shape)
  return tf.Variable(initial)

In [13]:
## Convolution and Pooling
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

** Graph w/ max pooling**

In [83]:
batch_size = 16
patch_size = 5
depth = 6
depth2 = 16
num_hidden = 256

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = weight_variable([patch_size, patch_size, num_channels, depth])
  layer1_biases = bias_variable([depth], 0.0)
  layer2_weights = weight_variable([patch_size, patch_size, depth, depth2])
  layer2_biases = bias_variable([depth2])
  ## fc layer
  layer3_weights = weight_variable([image_size / 4 * image_size / 4 * depth2, num_hidden])
  layer3_biases = bias_variable([num_hidden])
  ## readout layer
  layer4_weights = weight_variable([num_hidden, num_labels])
  layer4_biases = bias_variable([num_labels])

  ## Dropout
  ##keep_prob = tf.placeholder(tf.float32)
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    hidden = max_pool_2x2(hidden)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    hidden = max_pool_2x2(hidden)
    shape = hidden.get_shape().as_list()
    print shape
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    ##hidden = tf.nn.dropout(hidden, keep_prob)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  global_step = tf.Variable(0, trainable=False) # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.05, global_step, 200, 0.96, staircase=False)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  ##optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

[16, 7, 7, 16]
[10000, 7, 7, 16]
[18724, 7, 7, 16]


** Run w/ max pooling **

In [84]:
num_steps = 6001
def step_eval():
  for step in xrange(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 300 == 0):
      print global_step.eval(), learning_rate.eval()
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.2f%%" % accuracy(predictions, batch_labels)
      accuracy_valid = accuracy(valid_prediction.eval(), valid_labels)
      print "Validation accuracy: %.2f%%" % accuracy_valid
      print "---------------------------"
      if accuracy_valid > 89.5:
        print "Halted!"
        return

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized\n==============="
  step_eval()
  print "Test accuracy: %.2f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
1 0.0499898
Minibatch loss at step 0 : 2.26451
Minibatch accuracy: 6.25%
Validation accuracy: 13.08%
---------------------------
301 0.0470206
Minibatch loss at step 300 : 1.09354
Minibatch accuracy: 75.00%
Validation accuracy: 80.98%
---------------------------
601 0.0442278
Minibatch loss at step 600 : 0.576989
Minibatch accuracy: 81.25%
Validation accuracy: 83.42%
---------------------------
901 0.0416008
Minibatch loss at step 900 : 0.210984
Minibatch accuracy: 93.75%
Validation accuracy: 83.93%
---------------------------
1201 0.0391299
Minibatch loss at step 1200 : 0.812855
Minibatch accuracy: 81.25%
Validation accuracy: 85.02%
---------------------------
1501 0.0368057
Minibatch loss at step 1500 : 0.328062
Minibatch accuracy: 87.50%
Validation accuracy: 85.57%
---------------------------
1801 0.0346196
Minibatch loss at step 1800 : 0.0465552
Minibatch accuracy: 100.00%
Validation accuracy: 86.20%
---------------------------
2101 0.0325634
Minibatch loss at step 2100

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

http://cs231n.github.io/convolutional-networks/#architectures

http://yann.lecun.com/exdb/publis/pdf/lecun-98.pdf

INPUT 28x28 -> <br>
C1 6@28x28 -> S2 6@14x14 -> <br>
C3 16@10x10 -> S4 16@5x5 -> <br>
C5 layer 120 -> F6 layer 84 -> <br>
OUPUT 10

    6001 0.00187146
    Minibatch loss at step 6000 : 0.6133
    Minibatch accuracy: 81.25%
    Validation accuracy: 87.17%
    Test accuracy: 93.48%
    
---

** LeNet-5 ** 

In [63]:
num_steps = 12499
def step_eval():
  for step in xrange(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 300 == 0):
      print global_step.eval(), learning_rate.eval()
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.2f%%" % accuracy(predictions, batch_labels)
      accuracy_valid = accuracy(valid_prediction.eval(), valid_labels)
      print "Validation accuracy: %.2f%%" % accuracy_valid
      print "---------------------------"

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized\n==============="
  step_eval()
  print "Test accuracy: %.2f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
1 0.0399796
Minibatch loss at step 0 : 2.49925
Minibatch accuracy: 6.25%
Validation accuracy: 10.10%
---------------------------
301 0.0343048
Minibatch loss at step 300 : 1.14317
Minibatch accuracy: 75.00%
Validation accuracy: 78.25%
---------------------------
601 0.0294356
Minibatch loss at step 600 : 0.741588
Minibatch accuracy: 75.00%
Validation accuracy: 81.18%
---------------------------
901 0.0252575
Minibatch loss at step 900 : 0.25126
Minibatch accuracy: 87.50%
Validation accuracy: 82.78%
---------------------------
1201 0.0216724
Minibatch loss at step 1200 : 0.936555
Minibatch accuracy: 68.75%
Validation accuracy: 84.21%
---------------------------
1501 0.0185962
Minibatch loss at step 1500 : 0.334108
Minibatch accuracy: 87.50%
Validation accuracy: 84.97%
---------------------------
1801 0.0159566
Minibatch loss at step 1800 : 0.0223163
Minibatch accuracy: 100.00%
Validation accuracy: 85.34%
---------------------------
2101 0.0136917
Minibatch loss at step 2100 

---
MNIST data
---------
https://www.tensorflow.org/versions/master/tutorials/mnist/pros/index.html#build-a-multilayer-convolutional-network

---

In [22]:
import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
